In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import requests

dataset_url = "https://www.philippe-fournier-viger.com/spmf/datasets/retail.txt"
data = requests.get(dataset_url).text.splitlines()
transactions = [line.split() for line in data]

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

min_support_values = [0.01, 0.02, 0.03]
frequent_patterns = {}

for min_support in min_support_values:
    frequent_patterns[min_support] = apriori(df, min_support=min_support, use_colnames=True)
    print(f"Frequent Patterns for min_support = {min_support}:")
    print(frequent_patterns[min_support])


In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from itertools import combinations

chosen_support = 0.01
frequent_itemsets = apriori(df, min_support=chosen_support, use_colnames=True)

min_confidence_values = [0.5, 0.6, 0.7]
association_rules_by_confidence = {}

for min_conf in min_confidence_values:
    rules = []
    for _, row in frequent_itemsets.iterrows():
        itemset = set(row['itemsets'])
        for item in itemset:
            antecedent = frozenset([item])
            consequent = itemset - antecedent
            if consequent:
                antecedent_support = frequent_itemsets[
                    frequent_itemsets['itemsets'] == antecedent
                ]['support'].values[0]
                rule_confidence = row['support'] / antecedent_support
                if rule_confidence >= min_conf:
                    rules.append({
                        'antecedent': antecedent,
                        'consequent': consequent,
                        'support': row['support'],
                        'confidence': rule_confidence
                    })
    association_rules_by_confidence[min_conf] = pd.DataFrame(rules)

for min_conf, rules_df in association_rules_by_confidence.items():
    print(f"\nAssociation Rules for min_confidence = {min_conf}:")
    print(rules_df)


In [ ]:
import matplotlib.pyplot as plt

support_counts = [len(frequent_patterns[support]) for support in min_support_values]
plt.figure(figsize=(10, 5))
plt.plot(min_support_values, support_counts, marker='o', color='b', label="Frequent Patterns")
plt.xlabel("Minimum Support")
plt.ylabel("Number of Frequent Patterns")
plt.title("Frequent Patterns vs. Minimum Support")
plt.legend()
plt.show()

conf_rule_counts = [
    len(association_rules_by_confidence[conf]) if conf in association_rules_by_confidence else 0
    for conf in min_confidence_values
]
plt.figure(figsize=(10, 5))
plt.plot(min_confidence_values, conf_rule_counts, marker='o', color='r', label="Association Rules")
plt.xlabel("Minimum Confidence")
plt.ylabel("Number of Association Rules")
plt.title("Association Rules vs. Minimum Confidence")
plt.legend()
plt.show()
